## 任务一
* Task01：线性回归；Softmax与分类模型、多层感知机（1天）
* Task02：文本预处理；语言模型；循环神经网络基础（1天）

### 目标
* 1.从线性回归模型入手理解用机器学习模型解决实际问题的通用逻辑（数据、模型、算法）
* 2.从回归问题怎样过渡到分类问题？
* 3.多层感知机--基础版神经网络？
* 4.文本类数据的预处理流程？
* 5.统计语言模型 VS 神经网络语言模型？


In [ ]:
## 盲点置顶
1. numpy里面二维array和matrix的区别

## 线性回归模型的pytorch实现

导包--》数据准备--》定义模型--》模型训练--》模型预测与评估

In [7]:
#导包
import torch
from torch import nn
import numpy as np
torch.manual_seed(1)  #随机数种子定为1
import random
print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')

1.4.0


### 数据准备
使用线性模型来生成数据集，生成一个1000个样本的数据集，下面是用来生成数据的线性关系：

$$
\mathrm{price} = w_{\mathrm{area}} \cdot \mathrm{area} + w_{\mathrm{age}} \cdot \mathrm{age} + b
$$


In [4]:
#生成数据
# set input feature number 
num_inputs = 2
# set example number
num_examples = 1000

# set true weight and bias in order to generate corresponded label
true_w = [2, -3.4]
true_b = 4.2

features = torch.randn(num_examples, num_inputs,
                      dtype=torch.float32)   #randn()函数返回1000*2的矩阵
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()),
                       dtype=torch.float32)

In [5]:
#读取数据集
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)  # random read 10 samples
    for i in range(0, num_examples, batch_size):
        j = torch.LongTensor(indices[i: min(i + batch_size, num_examples)]) # the last time may be not enough for a whole batch
        yield  features.index_select(0, j), labels.index_select(0, j)

In [8]:
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

tensor([[ 0.3325,  1.5443],
        [ 0.3705,  0.8323],
        [ 0.5828,  0.5643],
        [-0.5009,  0.7134],
        [ 0.3632, -0.3996],
        [-0.2719,  0.6688],
        [-0.8170, -0.2216],
        [-1.2404, -1.3484],
        [-1.1910,  1.3271],
        [-0.8030,  0.6995]]) 
 tensor([-0.3887,  2.1274,  3.4469,  0.7859,  6.2746,  1.3716,  3.3155,  6.3062,
        -2.6998,  0.2120])


### 初始化模型参数

In [9]:
w = torch.tensor(np.random.normal(0, 0.01, (num_inputs, 1)), dtype=torch.float32)
b = torch.zeros(1, dtype=torch.float32)

w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

tensor([0.], requires_grad=True)

### 定义模型
定义用来训练参数的训练模型：

$$
\mathrm{price} = w_{\mathrm{area}} \cdot \mathrm{area} + w_{\mathrm{age}} \cdot \mathrm{age} + b
$$



In [10]:
def linreg(X, w, b):
    return torch.mm(X, w) + b

### 定义损失函数
我们使用的是均方误差损失函数：
$$
l^{(i)}(\mathbf{w}, b) = \frac{1}{2} \left(\hat{y}^{(i)} - y^{(i)}\right)^2,
$$


In [11]:
def squared_loss(y_hat, y): 
    return (y_hat - y.view(y_hat.size())) ** 2 / 2

### 定义优化函数
在这里优化函数使用的是小批量随机梯度下降：

$$
(\mathbf{w},b) \leftarrow (\mathbf{w},b) - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}} \partial_{(\mathbf{w},b)} l^{(i)}(\mathbf{w},b)
$$
  

In [12]:
def sgd(params, lr, batch_size): 
    for param in params:
        param.data -= lr * param.grad / batch_size # ues .data to operate param without gradient track

### 训练
当数据集、模型、损失函数和优化函数定义完了之后就可来准备进行模型的训练了。


In [13]:
# super parameters init
lr = 0.03
num_epochs = 5

net = linreg
loss = squared_loss

# training
for epoch in range(num_epochs):  # training repeats num_epochs times
    # in each epoch, all the samples in dataset will be used once
    
    # X is the feature and y is the label of a batch sample
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y).sum()  
        # calculate the gradient of batch sample loss 
        l.backward()  
        # using small batch random gradient descent to iter model parameters
        sgd([w, b], lr, batch_size)  
        # reset parameter gradient
        w.grad.data.zero_()
        b.grad.data.zero_()
    train_l = loss(net(features, w, b), labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().item()))

epoch 1, loss 0.026165
epoch 2, loss 0.000095
epoch 3, loss 0.000051
epoch 4, loss 0.000051
epoch 5, loss 0.000051
